In [9]:
import sys
sys.path.append('/Users/joachim/texjs/lva/IntroSC/ASC-ODE/build/mechsystem')
sys.path.append('../build/mechsystem')

from mass_spring import *

from pythreejs import *

In [ ]:
mss = MassSpringSystem3d()
mss.gravity = (0,-9.81,0)

#mA = mss.add (Mass(1, (1,0,0)))
#mB = mss.add (Mass(2, (2,0,0)))
#f1 = mss.add (Fix( (0,0,0)) )
#mss.add (Spring(1, 200000, (f1, mA)))
##mss.add (DistanceConstraint(1, (f1, mA)))
#mss.add (Spring(1, 100000, (mA, mB)))

# Simplified variant for testing with Constraints
mA = mss.add (Mass(1, (0,0,1)))
f1 = mss.add (Fix( (0,0,0)) )
mss.add (Spring(1, 20, (f1, mA)))
#mss.addConstraint (f1, mA)

0

In [11]:
masses = []
for m in mss.masses:
    masses.append(
        Mesh(SphereBufferGeometry(0.2, 16, 16),
             MeshStandardMaterial(color='red'),
             position=m.pos)) 

fixes = []
for f in mss.fixes:
    fixes.append(
        Mesh(SphereBufferGeometry(0.2, 32, 16),
             MeshStandardMaterial(color='blue'),
             position=f.pos)) 

springpos = []
springs = None
for s in mss.springs:
    pA = mss[s.connectors[0]].pos
    pB = mss[s.connectors[1]].pos
    print( "Spring between ", pA, " and ", pB)
    springpos.append ([ pA, pB ] ) 
if springpos:
    springgeo = LineSegmentsGeometry(positions=springpos)
    m2 = LineMaterial(linewidth=3, color='cyan')
    springs = LineSegments2(springgeo, m2)    

constraintpos = []
constraints = None
for c in mss.distanceConstraints:
    pA = mss[c.connectors[0]].pos
    pB = mss[c.connectors[1]].pos
    print( "Constraint between ", pA, " and ", pB)
    constraintpos.append ([ pA, pB ] ) 
if constraintpos:
    constraintgeo = LineSegmentsGeometry(positions=constraintpos)
    m3 = LineMaterial(linewidth=3, color='orange')
    constraints = LineSegments2(constraintgeo, m3)

axes = AxesHelper(1)

Spring between  [0.0, 0.0, 0.0]  and  [0.0, 0.0, 1.0]


In [12]:
view_width = 600
view_height = 400

camera = PerspectiveCamera( position=[10, 6, 10], aspect=view_width/view_height)
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()

scene = Scene(children=[*masses, *fixes, springs, constraints, axes, camera, key_light, ambient_light])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)

renderer

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(10.0, 6.0, 10.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0…

In [13]:
from time import sleep
for i in range(10000):
    mss.simulate (0.02, 100)
    for m,mvis in zip(mss.masses, masses):
        mvis.position = (m.pos[0], m.pos[1], m.pos[2])

    springpos = []
    for s in mss.springs:
        pA = mss[s.connectors[0]].pos
        pB = mss[s.connectors[1]].pos
        springpos.append ([ pA, pB ]) 
    if springpos:
        springs.geometry = LineSegmentsGeometry(positions=springpos)
    constraintpos = []
    for c in mss.distanceConstraints:
        pA = mss[c.connectors[0]].pos
        pB = mss[c.connectors[1]].pos
        constraintpos.append ([ pA, pB ] ) 
    if constraintpos:
        constraints.geometry = LineSegmentsGeometry(positions=constraintpos)
    sleep(0.01)

KeyboardInterrupt: 